In [46]:
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,precision_score,recall_score
from collections import Counter
import pandas as pd
import numpy as np

''' Summary of the file.
    
    功能：
        读取特征值文件和标签文件，通过交叉验证获得训练集和测试集，以随机森林方法划分并获得准确率，通过混淆矩阵标示结果。
    
    输出：
        结果的混淆矩阵，每次划分的准确率，十次的平均准确率
'''



' Summary of the file.\n    \n    功能：\n        读取特征值文件和标签文件，通过交叉验证获得训练集和测试集，以随机森林方法划分并获得准确率，通过混淆矩阵标示结果。\n    \n    输出：\n        结果的混淆矩阵，每次划分的准确率，十次的平均准确率\n'

In [47]:
# 实验室的15000条恶意样本，分为十个家族


# LBP_block
subfeatures=pd.read_csv(r'E:\test\process_lpb_block2\lbp_kmeans_hist_feature.csv',header=None)
labels = pd.read_csv(r'E:\test\process_lpb_block2\ClassNo_full.txt',header=None)



"\n# # GIST特征值\nsubfeatures=pd.read_csv(r'E:\test\\process_gist\\gist_f_train_full.csv',header=None)\nlabels = pd.read_csv(r'E:\test\\process_gist\\CNO_full.txt',header=None)\n"

In [48]:
# 把先提取的家族10的特征挪到最后
result = pd.concat([subfeatures, labels], axis=1,ignore_index=True)
# print(result)
result2 = result.iloc[:1431]
list = []
for i in range(len(result)):
    if result.iloc[i,-1] == 10:
        list.append(i)
result = result.drop(list)
result = pd.concat([result,result2],axis=0,ignore_index=True)
subfeatures = result.iloc[:,:-1]
labels = result.iloc[:,-1]

In [49]:
# 平均准确率归零
avgscore = 0
recallscore = 0
precisionscore = 0
N = 10




In [50]:
# 进行十次随机森林测试
for i in range(N):
    # 以10%的比例进行交叉验证
    # X_train, X_test, y_train, y_test = cross_validation.train_test_split(subfeatures,features_labels,test_size=0.1)
    X_train, X_test, y_train, y_test = train_test_split(subfeatures, labels, test_size=0.1)

    # 进行训练
    print('train...')
    # 进行随机森林训练,30课树，不限制进程数
    srf = RF(n_estimators=30, n_jobs=-1)
    srf.fit(X_train, y_train)

    # 预试
    print("test...")
    c_test = srf.predict(X_test)


    # 计算预测划分准确率
    score = srf.score(X_test, y_test)
    print(score)
    print(classification_report(y_test,c_test))
    # print("c_test")
    # print(c_test)
    # print('y_test')
    # print(y_test)

    avgscore = avgscore + score
    recallscore = recallscore + recall_score(y_test,c_test,average="macro")
    precisionscore = precisionscore + precision_score(y_test,c_test,average="macro")
    print(" initial test  labels size")

    # 通过混淆矩阵进行结果标示
    cm = confusion_matrix(y_test, c_test)
    np.set_printoptions(threshold=10000)
    np.set_printoptions(precision=2)
    print('Confusion matrix, without normalization')
    print(str(cm))


train...
test...
0.8722433460076046
              precision    recall  f1-score   support

           1       0.54      0.62      0.58       115
           2       0.89      0.85      0.87       124
           3       0.78      0.71      0.74       103
           4       0.83      0.99      0.90       150
           5       0.98      0.99      0.98       161
           6       1.00      0.99      1.00       158
           7       0.84      0.93      0.88       131
           8       1.00      0.94      0.97        63
           9       0.95      0.74      0.83       164
          10       0.93      0.90      0.92       146

   micro avg       0.87      0.87      0.87      1315
   macro avg       0.87      0.87      0.87      1315
weighted avg       0.88      0.87      0.87      1315

 initial test  labels size
Confusion matrix, without normalization
[[ 71   7   7  14   0   0   5   0   3   8]
 [ 13 105   6   0   0   0   0   0   0   0]
 [ 14   5  73   3   4   0   2   0   1   1]
 [  0   0

test...
0.8912547528517111
              precision    recall  f1-score   support

           1       0.72      0.63      0.67       138
           2       0.93      0.84      0.89       166
           3       0.78      0.82      0.80       101
           4       0.86      0.99      0.92       138
           5       1.00      0.99      0.99       148
           6       1.00      0.98      0.99       131
           7       0.88      0.91      0.89       138
           8       0.90      1.00      0.95        63
           9       0.93      0.86      0.90       145
          10       0.87      0.94      0.90       147

   micro avg       0.89      0.89      0.89      1315
   macro avg       0.89      0.90      0.89      1315
weighted avg       0.89      0.89      0.89      1315

 initial test  labels size
Confusion matrix, without normalization
[[ 87   2  11   9   0   0   7   2   6  14]
 [ 16 140   6   0   0   0   1   3   0   0]
 [  7   3  83   2   0   0   1   2   1   2]
 [  0   0   0 136 

In [51]:
# 输出N次的平均准确率
avgscore = avgscore / N
recallscore = recallscore / N
precisionscore = precisionscore / N

print('avgscore....')
print(avgscore)
print('False positive rate')
print(1-precisionscore)
print('false negative rate')
print(1-recallscore)

avgscore....
0.8880608365019012
False positive rate
0.11197718078396157
false negative rate
0.11586092345307719
